# Language Model Programming

Use the corpus `std/de/de-universal-train.conll` from the data set Universal Treebanks to train a Bigram model with Laplace smoothing. (Feel free to try other models if you like.)

Determine the perplexity of the model on the first 100 sentences from the corpus `std/de/de-universal-test.conll`.

*Note*: It is sufficient to use the first 10000 sentences for training. For larger ngrams and more sentences the training gets computationally expensive so be careful with these hyperparameters!<br>
*Note*: to enable unknown contexts, run `lm.counts.update([[('<UNK>', '<UNK>')]])` after training.

In [2]:
from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends, flatten
from nltk.lm import MLE, Laplace, KneserNeyInterpolated, Vocabulary
from nltk.util import ngrams
from functools import reduce
import math

In [3]:
# hyperparameters
n = 2
k = 10000

In [4]:
from nltk.corpus import universal_treebanks as ut
test_sentences = ut.sents("std/de/de-universal-test.conll")[:100]
training_sentences = list(ut.sents("std/de/de-universal-train.conll")[:k])
test_sentences[:3],training_sentences[:3]

([['Der', 'Hauptgang', 'war', 'in', 'Ordnung', ',', 'aber', 'alles', 'andere', 'als', 'umwerfend', '.'], ['Ich', 'habe', 'dort', '2007', 'meinen', 'OWD', 'gemacht', 'und', 'weil', 'mir', 'das', 'Tauchen', 'so', 'gefiel', 'hab', 'ich', 'dort', 'noch', 'im', 'selben', 'Jahr', 'den', 'AOWD', 'und', 'den', 'Deep', 'drangehängt', '.'], ...],
 [['Sehr',
   'gute',
   'Beratung',
   ',',
   'schnelle',
   'Behebung',
   'der',
   'Probleme',
   ',',
   'so',
   'stelle',
   'ich',
   'mir',
   'Kundenservice',
   'vor',
   '.'],
  ['Die', 'Kosten', 'sind', 'definitiv', 'auch', 'im', 'Rahmen', '.'],
  ['Nette', 'Gespräche', ',', 'klasse', 'Ergebnis']])

In [5]:
generate_ngrams= lambda n, x: sum((list(x) for x in list(padded_everygram_pipeline(n, x)[0])),[])

In [6]:
train, vocab = padded_everygram_pipeline(n,training_sentences)
lm = Laplace(n)
lm.fit(train, vocab)
lm.counts.update([[('<UNK>', '<UNK>')]])

In [7]:
test_dataset = generate_ngrams(n, test_sentences)
test_dataset

[('<s>',),
 ('<s>', 'Der'),
 ('Der',),
 ('Der', 'Hauptgang'),
 ('Hauptgang',),
 ('Hauptgang', 'war'),
 ('war',),
 ('war', 'in'),
 ('in',),
 ('in', 'Ordnung'),
 ('Ordnung',),
 ('Ordnung', ','),
 (',',),
 (',', 'aber'),
 ('aber',),
 ('aber', 'alles'),
 ('alles',),
 ('alles', 'andere'),
 ('andere',),
 ('andere', 'als'),
 ('als',),
 ('als', 'umwerfend'),
 ('umwerfend',),
 ('umwerfend', '.'),
 ('.',),
 ('.', '</s>'),
 ('</s>',),
 ('<s>',),
 ('<s>', 'Ich'),
 ('Ich',),
 ('Ich', 'habe'),
 ('habe',),
 ('habe', 'dort'),
 ('dort',),
 ('dort', '2007'),
 ('2007',),
 ('2007', 'meinen'),
 ('meinen',),
 ('meinen', 'OWD'),
 ('OWD',),
 ('OWD', 'gemacht'),
 ('gemacht',),
 ('gemacht', 'und'),
 ('und',),
 ('und', 'weil'),
 ('weil',),
 ('weil', 'mir'),
 ('mir',),
 ('mir', 'das'),
 ('das',),
 ('das', 'Tauchen'),
 ('Tauchen',),
 ('Tauchen', 'so'),
 ('so',),
 ('so', 'gefiel'),
 ('gefiel',),
 ('gefiel', 'hab'),
 ('hab',),
 ('hab', 'ich'),
 ('ich',),
 ('ich', 'dort'),
 ('dort',),
 ('dort', 'noch'),
 ('noch',),
 

In [8]:
lm.perplexity(test_dataset)

3475.8927714544216

## Exercise 2

Write a function `complete_gap(lm, sent, alts)`  that fills a gap in a sentence:

- `lm` is supposed to be a language model,
- `sent` is a sentence (as a list of tokens) with `None` at the position of the gap, 
- `alts` is a list of alternatives that can be used instead of None.
 
 The function should return the completed sentence. 
 It is helpful to first write  a function that calculates the sentence's score.

In [9]:
def sent_score(lm , sentence):
    ngrs = generate_ngrams(lm.order, [[x for x in sentence]])
    score = reduce(lambda x,y: x*y, [lm.score(ng[-1], ng[:-1]) for ng in ngrs])
    return score

In [10]:
def complete_gap(lm, sent, alts):
    """
    The solution is calculating the probability of the sentences with the words inserted and then 
    """
    
    all_scores = [] # Save scores
    gap_index = sent.index(None) # list index of None
    n = lm.order

    for a in alts:
        new_sent = sent[:gap_index] + [a] + sent[gap_index+1:]
        all_scores.append((new_sent, sent_score(lm, new_sent)))
    return max(all_scores, key= lambda x:x[1])    

    

In [11]:
complete_gap(lm, ['Das', 'ist', 'ein', 'Satz', 'mit', None, 'Lücke', '.'], 
             ['ein', 'eine', 'einer', 'einen'])

(['Das', 'ist', 'ein', 'Satz', 'mit', 'einer', 'Lücke', '.'],
 1.256406248798569e-56)

In [12]:
complete_gap(lm, ['Der', 'Eingang', None, 'sich', 'auf', 'der', 'linken', 'Seite', '.'], 
             ['findet', 'befindet', 'erfindet', 'ist'])

(['Der', 'Eingang', 'befindet', 'sich', 'auf', 'der', 'linken', 'Seite', '.'],
 1.3763181609651516e-63)

In [13]:
complete_gap(lm, ['Ich', 'habe', 'ein', None, 'Problem', '.'], 
             ['große', 'großes', 'großer', 'großen', 'größer'])

(['Ich', 'habe', 'ein', 'großer', 'Problem', '.'], 1.065553837001718e-45)

Not solvable from the corpus: The combination of "großes Problem" does not occurr so it is decided by the probability of "ein großes", which is less often than "ein großer":

In [14]:
lm.counts[["großes"]]["Problem"], lm.counts[["ein"]]["großes"], lm.counts[["ein"]]["großer"]


(0, 4, 4)

# Language Detection

We can use the language model's probability scores to classify languages. 
To classify language using language models, we calculate the probability of every language model for sentence and decide by maximum likelihood. (Use the Laplace language model.)
The algorithm would be as follows:

    1. Establish a language model for every language. (Be careful to use the same hyperparameters!)
    2. For a given input, calculate the language model probability
    3. Chose the language with the highest probability
    
To keep it comparable we use the same corpora from the previous language detection exercises.

In [16]:
import nltk
import pathlib
import matplotlib.pyplot as plt
from functools import reduce

number_of_words = 200000

english_text = nltk.corpus.gutenberg.words(nltk.corpus.gutenberg.fileids())[:number_of_words]
english_text = [x.lower() for x in english_text]
def load_german_text(path):
    text = []
    for f in pathlib.Path(path).glob("*.txt"):
        with open(f, "r") as openf:
            text += nltk.word_tokenize(openf.read().lower())
        if len(text) > number_of_words:
            break
    return text
german_text = load_german_text("../Woche 5/tagesschau_corpus")     
german_text[:10]

['david',
 's.',
 'tötete',
 '2016',
 'am',
 'münchner',
 'oez',
 'neun',
 'menschen',
 '.']

In [17]:
n=2
lms = {
    "german": Laplace(n), 
    "english":Laplace(n)
}

lms["german"].fit(*padded_everygram_pipeline(n, [german_text]))
lms["english"].fit(*padded_everygram_pipeline(n, [english_text]))

In [18]:
def identify_language(lms, sent):
    all_scores = []
    for language, lm in lms.items():
        n = lm.order
        all_scores.append((language, sent_score(lm, sent.lower().split(" "))))
    return max(all_scores, key= lambda x:x[1])[0]

In [19]:
identify_language(lms, "Ich arbeite an der Universität Leipzig.")

'german'

In [20]:
identify_language(lms, "I work at university Leipzig.")

'english'

In [21]:
identify_language(lms, "Ich heiße john"), identify_language(lms, "Ich heiße johannes")

('english', 'german')

In [25]:
identify_language(lms, "das"), identify_language(lms, "die")

('german', 'english')